In [1]:
import tensorflow as tf
import cv2
import numpy as np


2025-10-10 22:58:59.421359: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-10 22:58:59.486854: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-10 22:59:01.044894: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
CLIP_FRAMES = 32          # number of frames per clip (temporal length)
FRAME_SIZE = 112          # height & width of frames (112 or 128 are common)
BATCH_SIZE = 8
FPS = 25                  # approximate; used only if you need to compute frame counts
EPOCHS = 10
NUM_CLASSES= 2


In [3]:
import tensorflow as tf
print(tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))


2.20.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
import os
import glob
from sklearn.model_selection import train_test_split

# 1. Put your dataset folder path here
data_dir = "/home/rwt/Documents/team-preoject-cellula/Shop DataSet"

# 2. These must match your subfolder names
categories = ["shop lifters", "non shoplifters"]

file_label_list = []

# 3. Go through each folder and collect all .mp4 paths
for label, category in enumerate(categories):
    folder = os.path.join(data_dir, category)
    video_paths = glob.glob(os.path.join(folder, "*.mp4"))
    for path in video_paths:
        file_label_list.append((path, label))

# 4. Split dataset into train and validation sets
file_label_list_train, file_label_list_val = train_test_split(
    file_label_list,
    test_size=0.2,
    stratify=[label for _, label in file_label_list],
    random_state=42
)


In [5]:
import cv2
import numpy as np
import random
#clip_len = 32 is the number of frames i will take from each video to make a clip of 32 frames
# mode= 'random' is to Pick a random starting frame somewhere in the video
#if you don’t explicitly specify mode when calling the function,
#it will default to "random" automatically.
def sample_clip_from_video(video_path, clip_len=32, target_size=112, mode='start'):
    """
    mode: 'random' -> random temporal crop
          'center' -> center crop
          'all'    -> used only if total_frames == clip_len
    Returns: np.float32 array shape (clip_len, target_size, target_size, 3) normalized [0,1]
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        # fallback: read all frames into a list
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        total_frames = len(frames)
        cap.release()
        if total_frames == 0:
            raise IOError(f"No frames in {video_path}")
        # convert frames list to np.ndarray(4Darray (N,H,W,3))
        frames = [cv2.resize(f, (target_size, target_size)) for f in frames]
        frames = np.stack(frames, axis=0)
    else:
        # if total_frames >0 then set it to none for now later we will do its function
        frames = None

    if total_frames < clip_len:
        # If video shorter than clip_len -> loop frames or pad by repeating last frame
        # Simple approach: read all frames then repeat last
        if frames is None:
            frames = []
            cap = cv2.VideoCapture(video_path)
            while True:
                ret, frame = cap.read()
                if not ret: break
                frames.append(cv2.resize(frame, (target_size, target_size)))
            cap.release()
            frames = np.stack(frames, axis=0)
        # repeat last frames
        reps = clip_len - total_frames
        last = frames[-1:]
        frames = np.concatenate([frames, np.repeat(last, reps, axis=0)], axis=0)
        clip = frames[:clip_len]
    else:
        # choose start index
        if mode == 'random':
            start = random.randint(0, total_frames - clip_len)
        elif mode == 'center':
            start = max(0, (total_frames - clip_len) // 2)
        else:
            start = 0

        # read frames from start to start+clip_len-1
        clip = []
        cap.set(cv2.CAP_PROP_POS_FRAMES, start)
        read_count = 0
        while read_count < clip_len:
            ret, frame = cap.read()
            if not ret:
                # if reading fails unexpectedly, break and pad by last
                if len(clip) == 0:
                    raise IOError("Failed reading frames")
                last = clip[-1]
                clip.extend([last] * (clip_len - len(clip)))
                break
            frame = cv2.resize(frame, (target_size, target_size))
            clip.append(frame)
            read_count += 1
        clip = np.stack(clip, axis=0)  # (clip_len, H, W, 3)
        cap.release()

    # Convert BGR -> RGB and normalize to [0,1]
    clip = clip[..., ::-1].astype('float32') / 255.0
    return clip  # shape (clip_len, H, W, 3)


In [6]:
import tensorflow as tf

def video_clip_generator(file_label_list, clip_len=32, target_size=112, mode='random'):
    """
    file_label_list: list of tuples (video_path, label_index)
    Yields: (clip_np, label_index)
      clip_np shape: (clip_len, H, W, 3)
    """
    for video_path, label in file_label_list:
        clip = sample_clip_from_video(video_path, clip_len, target_size, mode=mode)
        yield clip, label

def make_dataset(file_label_list, clip_len=32, target_size=112, batch_size=8, shuffle=True, mode='random'):
    ds = tf.data.Dataset.from_generator(
        lambda: video_clip_generator(file_label_list, clip_len, target_size, mode),
        output_signature=(
            tf.TensorSpec(shape=(clip_len, target_size, target_size, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )
    )
    if shuffle:
        ds = ds.shuffle(buffer_size=len(file_label_list))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [7]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(e)


✅ GPU memory growth enabled


In [ ]:
#tf.config.set_visible_devices([], 'GPU')
from tensorflow import keras

from tensorflow.keras import layers, models

def conv3d_block(x, filters, kernel=(3,3,3), pool=(1,2,2)):
    x = layers.Conv3D(filters, kernel, padding='same', activation=None)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool3D(pool_size=pool, padding='same')(x)
    return x

def build_3dcnn(input_shape=(32,112,112,3), num_classes=10):
    inp = layers.Input(shape=input_shape)  # (T, H, W, C)
    x = conv3d_block(inp, 32, kernel=(3,3,3), pool=(1,2,2))
    x = conv3d_block(x, 64, kernel=(3,3,3), pool=(2,2,2))
    x = conv3d_block(x, 128, kernel=(3,3,3), pool=(2,2,2))
    x = conv3d_block(x, 256, kernel=(3,3,3), pool=(2,2,2))
    # Reduce spatial dims; keep some temporal info
    x = layers.GlobalAveragePooling3D()(x)  # result -> (batch, channels)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu')(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=inp, outputs=out)
    return model

# Example
model = build_3dcnn(input_shape=(CLIP_FRAMES, FRAME_SIZE, FRAME_SIZE, 3), num_classes=NUM_CLASSES)
model.summary()


I0000 00:00:1760126415.116494   37940 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4302 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-10-10 23:00:15.485581: W external/local_xla/xla/service/gpu/llvm_gpu_backend/default/nvptx_libdevice_path.cc:41] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykernel_launcher.runfiles/cuda_nvdisasm
  ipykernel_launcher.runfiles/nvidia_nvshmem
  ipykern/cuda_nvcc
  ipykern/cuda_nvdisasm
  ipykern/nvidia_nvshmem
  
  /usr/local/cuda
  /opt/cuda
  /home/rwt/Documents/myenv/lib/python3.12/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/rwt/Documents/myenv/lib/python3.12/site-packages

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 112, 112,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 32, 112, 112,   │         2,624 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 112, 112,   │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 32, 112, 112,   │             0 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 32, 56, 56, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 32, 56, 56, 64) │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 56, 56, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 32, 56, 56, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 16, 28, 28, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 16, 28, 28,     │       221,312 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 28, 28,     │           512 │
│ (BatchNormalization)            │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 16, 28, 28,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 8, 14, 14, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 8, 14, 14, 256) │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8, 14, 14, 256) │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 8, 14, 14, 256) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 4, 7, 7, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,79

 Total params: 1,232,514 (4.70 MB)

 Trainable params: 1,231,554 (4.70 MB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Suppose file_label_list_train and _val exist as lists of (path, label_index)
train_ds = make_dataset(file_label_list_train, clip_len=CLIP_FRAMES, target_size=FRAME_SIZE,
                        batch_size=BATCH_SIZE, mode='start')
val_ds = make_dataset(file_label_list_val, clip_len=CLIP_FRAMES, target_size=FRAME_SIZE,
                      batch_size=BATCH_SIZE, mode='start')

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_3dcnn.h5', save_best_only=True, monitor='val_loss'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
]
#removed shuffle , call backs but still the kernel dies
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)


Epoch 1/10


2025-10-10 23:00:37.302628: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:453] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 80 of 259
2025-10-10 23:00:57.382934: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:453] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 237 of 259
2025-10-10 23:01:00.203415: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:483] Shuffle buffer filled.
2025-10-10 23:01:00.256885: I external/local_xla/xla/service/service.cc:163] XLA service 0x7cec2c20a9b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-10 23:01:00.257004: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2025-10-10 23:01:00.348983: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-10

: 